## Installs for new environments

In [1]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 11.3 MB 1.1 MB/s eta 0:00:01    |███████████▌                    | 4.0 MB 1.1 MB/s eta 0:00:07
     |████████████████████████████████| 1.2 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 884 kB 77.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 39.3 MB 40.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Imports

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import math

import datetime


## Config

In [9]:
image_dim = 256 # eventual width/height of images
input_shape = (image_dim, image_dim, 3) #(width, height, RGB)

epochs = 16
batch_size = 32
learning_rate = .001

train_dir = 'dataset/train'
validate_dir = 'dataset/validate'

save_dir = 'models/not_augmented'
file_name_base = 'not_augmented'

base_log_dir = '/logs'

## Dataset Loading Augmentation

In [4]:
train = ImageDataGenerator(    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
) # Reinforcements for training dataset

validate = ImageDataGenerator() # Reinforcements for validation dataset

#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = validate.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset

Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## Dataset Loading: No Augmentation

In [10]:
train = ImageDataGenerator() # Reinforcements for training dataset

validate = ImageDataGenerator() # Reinforcements for validation dataset

#loads the datasets
train_dataset = train.flow_from_directory(train_dir, target_size=(image_dim, image_dim), class_mode='categorical') 
validate_dataset = validate.flow_from_directory(validate_dir, target_size=(image_dim, image_dim), class_mode='categorical')

# tf.keras.preprocessing.image_dataset_from_directory - this can be used for no reinforcement

n_train = train_dataset.samples # number of images in training dataset
n_validate = validate_dataset.samples # number of images in validation dataset


Found 8000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## CNN Design

In [11]:
class_keys = list(train_dataset.class_indices.keys())
n_classes = len(class_keys)

# defines the layers of the cnn
model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(n_classes, activation='softmax')
])

## Compilation

In [12]:
log_dir = base_log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
              loss='MSE',
              metrics=['accuracy'])

## Evaluate and Save the initial model

In [13]:
model.evaluate(validate_dataset, batch_size=256)
model.save(save_dir + '/' + file_name_base + '_0')

63/63 [==============================] - 6s 90ms/step - loss: 0.1273 - accuracy: 0.0920
INFO:tensorflow:Assets written to: models/not_augmented/not_augmented_0/assets


## Training

In [14]:
model = tf.keras.models.load_model(save_dir + '/' + file_name_base + '_32')

In [14]:
for i in range(0,4):
    print('Training until ' + str((i+1) * epochs) + ' epochs')
    model.fit(train_dataset,
                        steps_per_epoch=math.floor(n_train/batch_size),
                        validation_data=validate_dataset,
                        validation_steps=math.floor(n_validate/batch_size),
                        epochs=epochs,
                        callbacks=[tensorboard])
    model.save(save_dir + '/' + file_name_base + '_' + str((i+1)*epochs))


Training until 16epochs
Epoch 1/16
250/250 [==============================] - 31s 121ms/step - loss: 0.0895 - accuracy: 0.1941 - val_loss: 0.0831 - val_accuracy: 0.2732
Epoch 2/16
250/250 [==============================] - 29s 116ms/step - loss: 0.0809 - accuracy: 0.3079 - val_loss: 0.0808 - val_accuracy: 0.3024
Epoch 3/16
250/250 [==============================] - 29s 117ms/step - loss: 0.0774 - accuracy: 0.3599 - val_loss: 0.0790 - val_accuracy: 0.3412
Epoch 4/16
250/250 [==============================] - 30s 120ms/step - loss: 0.0744 - accuracy: 0.3950 - val_loss: 0.0754 - val_accuracy: 0.3770
Epoch 5/16
250/250 [==============================] - 28s 112ms/step - loss: 0.0711 - accuracy: 0.4310 - val_loss: 0.0744 - val_accuracy: 0.3982
Epoch 6/16
250/250 [==============================] - 30s 119ms/step - loss: 0.0686 - accuracy: 0.4585 - val_loss: 0.0740 - val_accuracy: 0.4062
Epoch 7/16
250/250 [==============================] - 30s 121ms/step - loss: 0.0661 - accuracy: 0.4871 - v

250/250 [==============================] - 30s 119ms/step - loss: 0.0046 - accuracy: 0.9686 - val_loss: 0.0786 - val_accuracy: 0.4889
Epoch 8/16
250/250 [==============================] - 30s 120ms/step - loss: 0.0047 - accuracy: 0.9688 - val_loss: 0.0793 - val_accuracy: 0.4733
Epoch 9/16
250/250 [==============================] - 30s 118ms/step - loss: 0.0044 - accuracy: 0.9701 - val_loss: 0.0793 - val_accuracy: 0.4743
Epoch 10/16
250/250 [==============================] - 29s 117ms/step - loss: 0.0043 - accuracy: 0.9704 - val_loss: 0.0777 - val_accuracy: 0.4945
Epoch 11/16
250/250 [==============================] - 30s 119ms/step - loss: 0.0042 - accuracy: 0.9705 - val_loss: 0.0774 - val_accuracy: 0.4980
Epoch 12/16
250/250 [==============================] - 29s 117ms/step - loss: 0.0041 - accuracy: 0.9706 - val_loss: 0.0772 - val_accuracy: 0.4995
Epoch 13/16
250/250 [==============================] - 30s 119ms/step - loss: 0.0041 - accuracy: 0.9714 - val_loss: 0.0781 - val_accuracy:

## Save Trained Model

In [19]:
model.save(save_dir + '/' + file_name_base + '_45')

INFO:tensorflow:Assets written to: models/reinforced_full/reinforced_full_45\assets
